# Instagram Media Downloader

This notebook provides an agentic way to download images and videos from Instagram posts using your authenticated session.

## Features
- Automated login to Instagram
- Navigate through posts
- Intercept network requests to capture media URLs
- Download images and videos

## Requirements
Install the required packages first:
```bash
pip install selenium requests pillow webdriver-manager
```

In [1]:
# Install required packages
!pip install selenium requests pillow webdriver-manager --quiet

In [2]:
import json
import os
import time
import requests
from datetime import datetime
from pathlib import Path
from urllib.parse import urlparse

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

## Configuration

In [3]:
# Configuration
DOWNLOAD_FOLDER = "instagram_downloads"
MOBILE_VIEW = True  # Toggle Chrome's device toolbar (Ctrl+Shift+M) to keep Instagram in mobile layout

# Create download folder
Path(DOWNLOAD_FOLDER).mkdir(exist_ok=True)

print(f"Downloads will be saved to: {os.path.abspath(DOWNLOAD_FOLDER)}")

Downloads will be saved to: d:\OneDrive - Emory\Schweidel\PerceptionMap\code\instagram_downloads


## Instagram Media Downloader Class

In [ ]:
class InstagramMediaDownloader:
    def __init__(self, mobile_view=True, headless=False, debug=False):
        self.download_folder = DOWNLOAD_FOLDER
        self.driver = None
        self.mobile_view = mobile_view
        self.headless = headless
        self.debug = debug  # Enable verbose logging to verify network capture
        self._devtools_opened = False

    def setup_driver(self):
        """Setup Chrome driver with appropriate options"""
        chrome_options = Options()

        if self.headless:
            chrome_options.add_argument('--headless=new')

        chrome_options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        chrome_options.add_argument('--disable-blink-features=AutomationControlled')
        chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

        service = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=service, options=chrome_options)
        self.driver.implicitly_wait(10)
        self.driver.set_window_size(1200, 900)

        print("✓ Driver setup complete")

    def _drain_performance_logs(self):
        """Clear accumulated performance logs to avoid stale entries"""
        if not self.driver:
            return
        try:
            self.driver.get_log('performance')
        except Exception:
            pass

    def _ensure_mobile_toolbar(self):
        """Toggle Chrome's device toolbar to emulate mobile view using Ctrl+Shift+M"""
        if not self.mobile_view or self._devtools_opened or not self.driver:
            return
        try:
            body = self.driver.find_element(By.TAG_NAME, "body")
            ActionChains(self.driver).move_to_element(body).click(body).perform()
            time.sleep(0.5)
            
            # Press F12 once to open DevTools (keep it open for visibility)
            ActionChains(self.driver).send_keys(Keys.F12).perform()
            time.sleep(1.5)
            
            # Press Ctrl+Shift+M to toggle device toolbar
            ActionChains(self.driver).key_down(Keys.CONTROL).key_down(Keys.SHIFT).send_keys('m').key_up(Keys.SHIFT).key_up(Keys.CONTROL).perform()
            time.sleep(1)
            
            self._devtools_opened = True
            print("✓ Mobile view enabled (DevTools open)")
            if self.debug:
                print("  → Check Network tab > Img and Media sub-tabs to verify captured URLs")
        except Exception as e:
            print(f"⚠ Could not enable mobile view: {e}")

    def _click_if_present(self, locator, timeout=5):
        """Click an element if it becomes clickable within timeout"""
        try:
            element = WebDriverWait(self.driver, timeout).until(
                EC.element_to_be_clickable(locator)
            )
            element.click()
            time.sleep(1)
            return True
        except Exception:
            return False

    def login(self, username, password):
        """Login to Instagram"""
        print("Logging in to Instagram...")
        self.driver.get("https://www.instagram.com/accounts/login/")
        time.sleep(3)
        
        self._ensure_mobile_toolbar()
        
        wait = WebDriverWait(self.driver, 20)
        try:
            # Handle cookie consent if present
            self._click_if_present(
                (By.XPATH, "//button[contains(text(), 'Allow') or contains(text(), 'Accept')]"),
                timeout=5
            )
            
            # Enter credentials
            username_input = wait.until(EC.presence_of_element_located((By.NAME, "username")))
            password_input = wait.until(EC.presence_of_element_located((By.NAME, "password")))
            
            username_input.send_keys(username)
            password_input.send_keys(password)
            password_input.send_keys(Keys.RETURN)
            
            time.sleep(5)
            
            # Handle post-login prompts
            self._click_if_present((By.XPATH, "//button[contains(text(), 'Not Now')]"), timeout=3)
            self._click_if_present((By.XPATH, "//button[contains(text(), 'Not now')]"), timeout=3)
            
            print("✓ Login successful")
            return True
            
        except Exception as exc:
            print(f"✗ Login failed: {str(exc)}")
            return False

    def navigate_to_post(self, post_url):
        """Navigate to a specific Instagram post"""
        print(f"Navigating to post: {post_url}")
        self.driver.get(post_url)
        time.sleep(3)
        self._ensure_mobile_toolbar()
        print("✓ Post loaded")

    def _wait_for_media_to_render(self, timeout=10):
        """Wait for post media elements to appear"""
        try:
            WebDriverWait(self.driver, timeout).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "article img, article video"))
            )
        except TimeoutException:
            pass

    def _is_desired_media(self, url, mime_type, response):
        """Check if a media URL is desired (excludes static assets, profile pics, etc.)"""
        if not url or ('image' not in mime_type and 'video' not in mime_type):
            return False
        
        # Must be from Instagram CDN
        if 'cdninstagram.com' not in url and 'fbcdn.net' not in url:
            return False
        
        # Exclude static assets (logos, icons, UI elements)
        if 'static.cdninstagram.com' in url:
            return False
        
        path = urlparse(url).path.lower()
        
        # Exclude common non-media assets
        excluded_tokens = ('profilepic', 'sprite', 'favicon', 'glyph', 'badge', 'logo', 'emoji')
        if any(token in path for token in excluded_tokens):
            return False
        
        # Size check for images (exclude small thumbnails/icons)
        headers = response.get('headers', {})
        content_length = headers.get('content-length') or headers.get('Content-Length')
        if content_length and 'video' not in mime_type:
            try:
                if int(content_length) < 35000:
                    return False
            except ValueError:
                pass
        
        return True

    def extract_media_urls_from_logs(self, debug_prefix=""):
        """Extract media URLs from browser performance logs"""
        if not self.driver:
            return []
        try:
            logs = self.driver.get_log('performance')
        except Exception:
            return []
        
        if self.debug:
            print(f"\n{debug_prefix}[DEBUG] Processing {len(logs)} performance log entries...")
        
        media_urls = []
        image_count = 0
        video_count = 0
        filtered_out = []
        
        for log_entry in logs:
            try:
                message = json.loads(log_entry['message'])['message']
            except (KeyError, json.JSONDecodeError):
                continue
            
            if message.get('method') != 'Network.responseReceived':
                continue
            
            response = message.get('params', {}).get('response', {})
            url = response.get('url', '')
            mime_type = response.get('mimeType', '')
            
            # Debug: Show all image/video responses found
            if self.debug and url and ('image' in mime_type or 'video' in mime_type):
                filename = url.split('/')[-1].split('?')[0]
                if len(filename) > 50:
                    filename = filename[:50] + "..."
                
            if self._is_desired_media(url, mime_type, response):
                media_type = 'video' if 'video' in mime_type else 'image'
                media_urls.append({
                    'url': url,
                    'type': media_type
                })
                if media_type == 'video':
                    video_count += 1
                else:
                    image_count += 1
                    
                if self.debug:
                    filename = url.split('/')[-1].split('?')[0]
                    if len(filename) > 50:
                        filename = filename[:50] + "..."
                    print(f"{debug_prefix}  ✓ ACCEPTED {media_type}: {filename}")
            else:
                # Track why it was filtered
                if self.debug and url and ('image' in mime_type or 'video' in mime_type):
                    reason = ""
                    if 'static.cdninstagram.com' in url:
                        reason = "static CDN"
                    elif 'cdninstagram.com' not in url and 'fbcdn.net' not in url:
                        reason = "not Instagram CDN"
                    else:
                        path = urlparse(url).path.lower()
                        excluded_tokens = ('profilepic', 'sprite', 'favicon', 'glyph', 'badge', 'logo', 'emoji')
                        for token in excluded_tokens:
                            if token in path:
                                reason = f"contains '{token}'"
                                break
                        if not reason:
                            headers = response.get('headers', {})
                            content_length = headers.get('content-length') or headers.get('Content-Length')
                            if content_length and 'video' not in mime_type:
                                try:
                                    if int(content_length) < 35000:
                                        reason = f"too small ({content_length} bytes)"
                                except ValueError:
                                    pass
                    if reason:
                        filename = url.split('/')[-1].split('?')[0]
                        if len(filename) > 40:
                            filename = filename[:40] + "..."
                        filtered_out.append((filename, reason))
        
        if self.debug:
            print(f"{debug_prefix}[DEBUG] Found {image_count} images, {video_count} videos")
            if filtered_out:
                print(f"{debug_prefix}[DEBUG] Filtered out {len(filtered_out)} items:")
                for filename, reason in filtered_out[:5]:  # Show first 5
                    print(f"{debug_prefix}  ✗ {filename} - {reason}")
                if len(filtered_out) > 5:
                    print(f"{debug_prefix}  ... and {len(filtered_out) - 5} more")
        
        return media_urls

    def _click_next_slide(self):
        """Click the Next button in a carousel post"""
        selectors = [
            (By.CSS_SELECTOR, "button[aria-label='Next']"),
            (By.XPATH, "//button[@aria-label='Next']"),
            (By.XPATH, "//button//*[name()='svg' and @aria-label='Next']/.."),
        ]
        for locator in selectors:
            try:
                button = WebDriverWait(self.driver, 2).until(
                    EC.element_to_be_clickable(locator)
                )
                button.click()
                time.sleep(2)
                return True
            except Exception:
                continue
        return False

    def _collect_carousel_media(self):
        """
        Collect media from carousel post by progressively clicking Next.
        Collects ALL media (images and videos) from network logs.
        Uses seen set to prevent duplicates.
        """
        collected = []
        seen = set()
        
        if self.debug:
            print("\n" + "="*60)
            print("STARTING MEDIA COLLECTION")
            print("="*60)
            print("Compare the URLs below with what you see in Network tab > Img/Media")
        
        # Wait for initial media to load
        self._wait_for_media_to_render()
        time.sleep(2)
        
        # Get initial media (first 1-2 items: current + possibly preloaded)
        if self.debug:
            print("\n--- INITIAL LOAD (after refresh) ---")
        initial_media = self.extract_media_urls_from_logs(debug_prefix="[INITIAL] ")
        for media in initial_media:
            if media['url'] not in seen:
                seen.add(media['url'])
                collected.append(media)
        
        print(f"Found {len(collected)} initial media item(s)")
        
        # Drain logs to start fresh for next iterations
        self._drain_performance_logs()
        
        # Click Next and collect new media progressively
        click_count = 0
        max_clicks = 50  # Safety limit to prevent infinite loops
        no_new_media_count = 0  # Track consecutive clicks with no new media
        
        while click_count < max_clicks:
            if not self._click_next_slide():
                print("No more Next button found")
                break
                
            click_count += 1
            self._wait_for_media_to_render()
            time.sleep(1)
            
            if self.debug:
                print(f"\n--- AFTER NEXT CLICK #{click_count} ---")
            
            # Collect only NEW media after this Next click
            current_media = self.extract_media_urls_from_logs(debug_prefix=f"[NEXT-{click_count}] ")
            new_count = 0
            
            for media in current_media:
                if media['url'] not in seen:
                    seen.add(media['url'])
                    collected.append(media)
                    new_count += 1
            
            if new_count > 0:
                print(f"After Next click {click_count}: Found {new_count} new media item(s)")
                no_new_media_count = 0  # Reset counter
            else:
                no_new_media_count += 1
                print(f"After Next click {click_count}: No new media found")
                
                # If we've clicked Next 3 times without finding new media, stop
                if no_new_media_count >= 3:
                    print("No new media for 3 consecutive clicks - stopping")
                    break
            
            # Drain logs after each collection
            self._drain_performance_logs()
        
        if click_count >= max_clicks:
            print(f"⚠ Reached maximum click limit ({max_clicks})")
        
        if self.debug:
            print("\n" + "="*60)
            print(f"COLLECTION COMPLETE: {len(collected)} total media items")
            print("="*60)
        
        return collected

    def _extract_likes_and_comments(self):
        """Extract likes and comments count from the current post"""
        likes = None
        comments = None
        
        try:
            # Extract likes - look for button with like text or specific patterns
            try:
                # Try to find likes in various formats
                like_patterns = [
                    "span.x1ypdohk.x1s688f.x2fvf9.xe9ewy2[role='button']",
                    "section a[href*='/liked_by/']",
                    "a[href*='/liked_by/'] span",
                ]
                
                for pattern in like_patterns:
                    elements = self.driver.find_elements(By.CSS_SELECTOR, pattern)
                    for elem in elements:
                        text = elem.text.strip()
                        # Look for numbers in the text
                        import re
                        numbers = re.findall(r'\d+', text)
                        if numbers:
                            likes = int(numbers[0])
                            break
                    if likes is not None:
                        break
            except Exception:
                pass
            
            # Extract comments - look for comment count patterns
            try:
                comment_patterns = [
                    "span.html-span.xdj266r.x14z9mp.xat24cr.x1lziwak.xexx8yu.xyri2b.x18d9i69.x1c1uobl.x1hl2dhg.x16tdsg8.x1vvkbs",
                    "span.xdj266r",
                    "ul li div span",
                ]
                
                for pattern in comment_patterns:
                    elements = self.driver.find_elements(By.CSS_SELECTOR, pattern)
                    for elem in elements:
                        text = elem.text.strip()
                        # Only consider pure numbers (not mixed text)
                        if text.isdigit():
                            potential_comments = int(text)
                            # Comments are usually larger than single digits
                            if potential_comments > 0:
                                comments = potential_comments
                                break
                    if comments is not None:
                        break
            except Exception:
                pass
                    
        except Exception as e:
            print(f"⚠ Could not extract likes/comments: {e}")
        
        return likes, comments

    def download_media(self, media_list, post_id=None, likes=None, comments=None):
        """Download media files"""
        if not media_list:
            print("No media found to download")
            return []
        
        downloaded_files = []
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Display post stats
        if likes is not None or comments is not None:
            stats = []
            if likes is not None:
                stats.append(f"{likes} likes")
            if comments is not None:
                stats.append(f"{comments} comments")
            print(f"Post stats: {', '.join(stats)}")
        
        if self.debug:
            print("\n" + "="*60)
            print("STARTING DOWNLOADS")
            print("="*60)
        
        for idx, media in enumerate(media_list):
            time.sleep(2)
            try:
                url = media['url']
                media_type = media['type']
                extension = 'mp4' if media_type == 'video' else 'jpg'
                
                if post_id:
                    filename = f"{post_id}_{idx+1}_{timestamp}.{extension}"
                else:
                    filename = f"media_{idx+1}_{timestamp}.{extension}"
                
                filepath = os.path.join(self.download_folder, filename)
                
                print(f"Downloading {media_type} {idx+1}/{len(media_list)}: {filename}")
                
                if self.debug:
                    url_preview = url[:80] + "..." if len(url) > 80 else url
                    print(f"  URL: {url_preview}")
                
                response = requests.get(url, stream=True, timeout=30)
                response.raise_for_status()
                
                with open(filepath, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                
                downloaded_files.append(filepath)
                print(f"✓ Downloaded: {filename}")
                
            except Exception as exc:
                print(f"✗ Failed to download {media.get('type', 'media')}: {str(exc)}")
        
        return downloaded_files

    def download_post(self, post_url):
        """Download all media from a single post"""
        self.navigate_to_post(post_url)
        
        # Drain old logs and refresh page to clear network tab
        self._drain_performance_logs()
        print("Refreshing page to clear network logs...")
        self.driver.refresh()
        time.sleep(3)
        
        self._ensure_mobile_toolbar()
        self._wait_for_media_to_render()
        
        if self.debug:
            print("\n[DEBUG] After refresh, check Network tab > Img/Media sub-tabs")
            print("[DEBUG] The filenames shown there should match what we capture below\n")
        
        # Extract likes and comments
        likes, comments = self._extract_likes_and_comments()
        
        # Collect media progressively by clicking Next
        media_items = self._collect_carousel_media()
        
        post_id = post_url.rstrip('/').split('/')[-1]
        # Remove query parameters from post_id
        if '?' in post_id:
            post_id = post_id.split('?')[0]
            
        print(f"\nTotal media found: {len(media_items)} file(s)")
        
        downloaded = self.download_media(media_items, post_id, likes, comments)
        return downloaded

    def download_multiple_posts(self, post_urls):
        """Download media from multiple posts"""
        all_downloads = []
        for index, url in enumerate(post_urls, 1):
            print(f"\n{'='*60}")
            print(f"Processing post {index}/{len(post_urls)}")
            print(f"{'='*60}")
            try:
                self._drain_performance_logs()
                downloaded = self.download_post(url)
                all_downloads.extend(downloaded)
                time.sleep(2)
            except Exception as exc:
                print(f"✗ Error processing post: {str(exc)}")
        return all_downloads

    def close(self):
        """Close the browser"""
        if self.driver:
            self.driver.quit()
            print("\n✓ Browser closed")

## Usage Example 1: Download from a Single Post

**Debug Mode**: Set `debug=True` to see detailed logging that shows:
- All URLs captured from performance logs
- Which URLs are accepted and which are filtered out (with reasons)
- This lets you verify the captured URLs match what you see in Network tab > Img/Media sub-tabs

In [ ]:
# Initialize the downloader with DEBUG MODE enabled
# Debug mode shows you exactly what URLs are being captured so you can verify
# they match what you see in DevTools Network tab > Img/Media sub-tabs
downloader = InstagramMediaDownloader(mobile_view=MOBILE_VIEW, headless=False, debug=True)
downloader.setup_driver()

# Login (replace with your credentials)
USERNAME = "your_username"  # Replace with your Instagram username
PASSWORD = "your_password"  # Replace with your Instagram password

if downloader.login(USERNAME, PASSWORD):
    # Download from a single post
    POST_URL = "https://www.instagram.com/p/DOfLxX1j11j/?img_index=1" # "https://www.instagram.com/p/POST_ID/"  # Replace with actual post URL
    
    downloaded_files = downloader.download_post(POST_URL)
    
    print(f"\n{'='*60}")
    print(f"Download Complete!")
    print(f"{'='*60}")
    print(f"Total files downloaded: {len(downloaded_files)}")
    for file in downloaded_files:
        print(f"  - {file}")

# Close the browser
downloader.close()

## Usage Example 2: Download from Multiple Posts

In [ ]:
# Initialize the downloader
downloader = InstagramMediaDownloader(mobile_view=MOBILE_VIEW, headless=False)
downloader.setup_driver()

# Login
USERNAME = "your_username"
PASSWORD = "your_password"

if downloader.login(USERNAME, PASSWORD):
    # List of post URLs to download
    POST_URLS = [
        "https://www.instagram.com/p/POST_ID_1/",
        "https://www.instagram.com/p/POST_ID_2/",
        "https://www.instagram.com/p/POST_ID_3/",
    ]
    
    downloaded_files = downloader.download_multiple_posts(POST_URLS)
    
    print(f"\n{'='*60}")
    print(f"All Downloads Complete!")
    print(f"{'='*60}")
    print(f"Total files downloaded: {len(downloaded_files)}")

# Close the browser
downloader.close()

## Advanced: Interactive Mode

Use this to manually navigate and download media interactively.

In [ ]:
# Interactive mode - browser stays open for manual navigation
downloader = InstagramMediaDownloader(mobile_view=MOBILE_VIEW, headless=False)
downloader.setup_driver()

# Login
USERNAME = "your_username"
PASSWORD = "your_password"

downloader.login(USERNAME, PASSWORD)

print("\nBrowser is open. Navigate to posts manually.")
print("When you're on a post you want to download, run the next cell.")

In [ ]:
# Download media from current page
current_url = downloader.driver.current_url
print(f"Current URL: {current_url}")

if '/p/' in current_url or '/reel/' in current_url:
    downloaded = downloader.download_post(current_url)
    print(f"\nDownloaded {len(downloaded)} file(s)")
else:
    print("Please navigate to a post first!")

In [ ]:
# Close browser when done
downloader.close()

## Tips and Notes

1. **Mobile View**: Mobile emulation is enabled by default as it often makes media extraction easier
2. **Network Logs**: The script captures network traffic via Chrome DevTools Protocol performance logs to find media URLs automatically
3. **Debug Mode**: Set `debug=True` when initializing the downloader to see detailed logs showing:
   - Exactly what URLs are being captured from performance logs
   - Which files are accepted and which are filtered out (with reasons like "too small", "static CDN", etc.)
   - Use this to verify the URLs match what you see in DevTools Network tab > Img/Media sub-tabs
4. **Rate Limiting**: Add delays between downloads to avoid being rate-limited by Instagram
5. **Login Sessions**: Your session is preserved during the browser lifetime
6. **Headless Mode**: Set `headless=True` if you don't need to see the browser
7. **File Naming**: Files are named with post ID and timestamp for easy organization

## How Performance Logs Work

The script uses `driver.get_log('performance')` which captures Chrome DevTools Protocol events, including:
- All network requests (same data shown in Network tab)
- Response headers, URLs, and MIME types
- This is equivalent to what you see in DevTools Network > Img/Media sub-tabs

Enable debug mode to verify that captured URLs match what you see in the Network tab!

## Troubleshooting

- If login fails, check your credentials
- If media isn't found, enable debug mode (`debug=True`) to see what's being captured
- Instagram may require 2FA - use interactive mode to handle this manually
- Some posts may have anti-scraping measures - this is normal
- If you see duplicate downloads, ensure logs are being drained properly (debug mode will help diagnose)

In [ ]:
# ⚠️ EDUCATIONAL EXAMPLE ONLY
# This script collects visible <img> sources from the current page.
# Do NOT use this to scrape or download Instagram content without permission.

from playwright.sync_api import sync_playwright
import os
import requests

URL = "https://www.instagram.com/p/DOfLxX1j11j/"
SAVE_DIR = "downloads"

os.makedirs(SAVE_DIR, exist_ok=True)

with sync_playwright() as p:
    browser = p.chromium.launch(headless=False)
    page = browser.new_page()
    page.goto(URL, timeout=60000)

    # Wait for images to load
    page.wait_for_selector("img", timeout=10000)
    images = page.query_selector_all("img")

    print(f"Found {len(images)} images on {URL}")
    for i, img in enumerate(images, 1):
        src = img.get_attribute("src")
        if not src or "data:" in src:
            continue
        print(f"{i}: {src}")
        try:
            filename = os.path.join(SAVE_DIR, f"image_{i}.jpg")
            with open(filename, "wb") as f:
                f.write(requests.get(src).content)
        except Exception as e:
            print(f"Error saving {src}: {e}")

    browser.close()
